In [ ]:
# instalação do Weights & Biases
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [ ]:
# instalação do pytest
!pip install pytest pytest-sugar

In [ ]:
# importação das bibliotecas
import wandb
import numpy as np
import pandas as pd

In [ ]:
# wandb login
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# criando um arquivo de test
%%file test_data.py

import pytest
import wandb
import pandas as pd
import numpy as np
import os
from PIL import Image

# inicializando uma run no wandb
run = wandb.init(project="Breast Cancer TCC", job_type="check_data")

@pytest.fixture(scope="session")
def data():
    artifact = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/label_data_clean:latest', type='clean_data').file()
    artifact2 = run.use_artifact('valmirfrancisco1/Breast Cancer TCC/img_data_clean:latest', type='clean_data').file()
    labels = pd.read_csv(artifact)
    data_img = np.load(artifact2)

    return labels, data_img


def test_data_length(data):
    # verificamos se temos a quantidade necessária de dados e se os dois artefatos têm o mesmo tamanho
    labels, data_img = data
    assert (len(labels) > 2000) and (len(data_img) > 2000) and (len(labels) == len(data_img))


def test_image_data_shape(data):
    # verifica se todos os valores do artefato data_img têm o mesmo formato
    labels, data_img = data
    for i in data_img:
        assert i.shape == data_img[0].shape


def test_label_data_types(data):
    # verifica se todos os valores do artefato de rótulos são do mesmo tipo
    labels, data_img = data
    for i in range(0, len(labels.columns)):
        assert type(labels.dtypes[i]) == type(labels.dtypes[0])


def test_img_data_types(data):
    # verifica se todos os valores do artefato data_img têm o mesmo tipo
    labels, data_img = data
    for i in range(len(data_img)):
        assert type(data_img[i]) == type(data_img[0])


def test_image_data_valid(data):
    # verifica se os dados de img_dat são uma imagem válida
    # com este teste não precisamos de uma verificação nula no img_data!
    labels, data_img = data
    for i in data_img:
        img = i*255
        img = img.astype(np.uint8)
        image = Image.fromarray(img)
        assert (image.size == (224, 224) and image.mode == 'RGB')


def test_label_range(data):
    # testa se os valores da categoria estão dentro de um intervalo de valores esperado
    labels, data_img = data
    binary_column = ["site_id", "cancer", "biopsy", "invasive", "implant", "difficult_negative_case"]
    for i in binary_column:
        if not labels[i].isin([0, 1]).all():
            return False
    return True


def test_null_values_from_labels(data):
    # testa se há dados faltantes no quadro de dados
    labels, data_img = data
    assert not labels.isnull().any().any()

def test_number_of_columns(data):
    # verificar quantidade de colunas do dataset
    labels, data_img = data
    assert labels.shape[1] == 12

def test_same_quantity_labels(data):
    # verifica se os valores 0 e 1 de cancer estão na mesma quantidade
    labels, data_img = data
    filter_1 = labels["cancer"] == 1
    filter_0 = labels["cancer"] == 0
    assert (labels[filter_1].shape[0] == labels[filter_0].shape[0])

    # finaliza a run
    run.finish()

Overwriting test_data.py


In [ ]:
# inicializa o teste
!pytest . -vv

Test session starts (platform: linux, Python 3.10.12, pytest 7.4.4, pytest-sugar 1.0.0)
cachedir: .pytest_cache
rootdir: /content
plugins: sugar-1.0.0, anyio-3.7.1
collected 9 items                                                                                  

 test_data.py::test_data_length ✓                                                     11% █▎        
 test_data.py::test_image_data_shape ✓                                                22% ██▎       
 test_data.py::test_label_data_types ✓                                                33% ███▍      
 test_data.py::test_img_data_types ✓                                                  44% ████▌     
 test_data.py::test_image_data_valid ✓                                                56% █████▋    
 test_data.py::test_label_range ✓                                                     67% ██████▋   
 test_data.py::test_null_values_from_labels ✓                                         78% ███████▊  
 test_data.py::test_number_o